<a href="https://colab.research.google.com/github/Kennethfargose/Viz-techinques-for-fashion_minst/blob/main/Aidm3_viz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import plotly.express as pex
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from matplotlib.offsetbox import AnnotationBbox, OffsetImage
from sklearn.pipeline import Pipeline
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.manifold import TSNE


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
d = pd.read_csv('/content/drive/MyDrive/fashion-mnist_train.csv')
d.head()

In [ ]:


X, y = d.iloc[:, 1:].to_numpy(), d.iloc[:, 0].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.39, random_state=40)


In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']



In [ ]:

def plot_digits(X, y, min_distance=0.05, images=None, figsize=(13, 10)):
    # Let's scale the input features so that they range from 0 to 1
    X_normalized = MinMaxScaler().fit_transform(X)
    # Now we create the list of coordinates of the digits plotted so far.
    # We pretend that one is already plotted far away at the start, to
    # avoid `if` statements in the loop below
    neighbors = np.array([[10., 10.]])
    # The rest should be self-explanatory
    plt.figure(figsize=figsize)
    cmap = mpl.cm.get_cmap("prism")
    digits = np.unique(y)
    for digit in digits:
        plt.scatter(X_normalized[y == digit, 0], X_normalized[y == digit, 1], c=[cmap(digit / 9)])
    plt.axis("off")
    ax = plt.gcf().gca()  # get current axes in current figure
    for index, image_coord in enumerate(X_normalized):
        closest_distance = np.linalg.norm(np.array(neighbors) - image_coord, axis=1).min()
        if closest_distance > min_distance:
            neighbors = np.r_[neighbors, [image_coord]]
            if images is None:
                plt.text(image_coord[0], image_coord[1], str(int(y[index])),
                         color=cmap(y[index] / 9), fontdict={"weight": "bold", "size": 16})
            else:
                image = images[index].reshape(28, 28)
                imagebox = AnnotationBbox(OffsetImage(image, cmap="binary"), image_coord)
                ax.add_artist(imagebox)

In [ ]:
X_pca = PCA(n_components=2, random_state=42).fit_transform(X_train)
plot_digits(X_pca, y_train)
plt.show()


In [ ]:


pca_lle = Pipeline([
    ("pca", PCA(n_components=0.94, random_state=40)),
    ("lle", LocallyLinearEmbedding(n_components=2, random_state=40)),
])
X_pca_lle = pca_lle.fit_transform(X_train)
plot_digits(X_pca_lle, y_train)
plt.show()


In [ ]:


pca_tsne = Pipeline([
    ("pca", PCA(n_components=0.94, random_state=40)),
    ("tsne", TSNE(n_components=2, random_state=40)),
])
X_pca_tsne = pca_tsne.fit_transform(X_train)
plot_digits(X_pca_tsne, y_train)
plt.show()


In [ ]:
from sklearn.decomposition import KernelPCA
from sklearn.manifold import TSNE

In [ ]:
k_pca_tsne = Pipeline([
    ("kpca", KernelPCA(n_components=2, kernel='poly', random_state=42)),
    ("tsne", TSNE(n_components=2, random_state=42)),
])
X_pca_tsne_k= k_pca_tsne.fit_transform(X_train)
plot_digits(X_pca_tsne_k, y_train)
plt.show()
